In [12]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
file_path = 'D:/software/0install package/VOSviewer'

In [13]:
data = pd.read_excel("D:/software/0install package/VOSviewer/web tab.xlsx")
df_wc = data[data.WC.str.contains('; ')] #筛选包含分隔符的数据，web of science是分号+空格,只有有分割符才代表是出现了两个及以上学科
print(len(data))
print(len(df_wc))#对比两个数字查看比例，大部分论文(接近50%)只有一个学科，所以学科共现不一定有意义

210
117


In [22]:
data.head()#数据示例

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA
0,J,"Chen, LC; Papandreou, G; Kokkinos, I; Murphy, ...",NaN,NaN,NaN,"Chen, Liang-Chieh; Papandreou, George; Kokkino...",NaN,NaN,DeepLab: Semantic Image Segmentation with Deep...,IEEE TRANSACTIONS ON PATTERN ANALYSIS AND MACH...,...,15,"Computer Science, Artificial Intelligence; Eng...",Computer Science; Engineering,FY2ZU,WOS:000426687100005,28463186.0,NaN,NaN,NaN,2020-02-21
1,J,"Kizilcec, RF; Perez-Sanagustin, M; Maldonado, JJ",NaN,NaN,NaN,"Kizilcec, Rene F.; Perez-Sanagustin, Mar; Mald...",NaN,NaN,Self-regulated learning strategies predict lea...,COMPUTERS & EDUCATION,...,16,"Computer Science, Interdisciplinary Applicatio...",Computer Science; Education & Educational Rese...,EF9GN,WOS:000390638800003,NaN,NaN,NaN,NaN,2020-02-21
2,J,"Wang, XY; Gao, LJ; Mao, SW; Pandey, S",NaN,NaN,NaN,"Wang, Xuyu; Gao, Lingjun; Mao, Shiwen; Pandey,...",NaN,NaN,CSI-Based Fingerprinting for Indoor Localizati...,IEEE TRANSACTIONS ON VEHICULAR TECHNOLOGY,...,14,"Engineering, Electrical & Electronic; Telecomm...",Engineering; Telecommunications; Transportation,EK8MM,WOS:000394178000062,NaN,NaN,NaN,NaN,2020-02-21
3,J,"Zhang, KP; Zhang, ZP; Li, ZF; Qiao, Y",NaN,NaN,NaN,"Zhang, Kaipeng; Zhang, Zhanpeng; Li, Zhifeng; ...",NaN,NaN,Joint Face Detection and Alignment Using Multi...,IEEE SIGNAL PROCESSING LETTERS,...,5,"Engineering, Electrical & Electronic",Engineering,DX3SN,WOS:000384293500003,NaN,NaN,NaN,NaN,2020-02-21
4,J,"Hare, S; Golodetz, S; Saffari, A; Vineet, V; C...",NaN,NaN,NaN,"Hare, Sam; Golodetz, Stuart; Saffari, Amir; Vi...",NaN,NaN,Struck: Structured Output Tracking with Kernels,IEEE TRANSACTIONS ON PATTERN ANALYSIS AND MACH...,...,14,"Computer Science, Artificial Intelligence; Eng...",Computer Science; Engineering,DX2YV,WOS:000384240600013,26700968.0,NaN,NaN,NaN,2020-02-21


In [14]:
xueke_list = []
for wc in df_wc.WC:
    wc_list = wc.split("; ")
    xueke_list.extend(wc_list)
print(len(xueke_list))
xueke_list_unique = list(set(xueke_list))
print(len(xueke_list_unique))

262
37


In [15]:
xueke_list_unique#web of science上边导出的学科包含二级学科，如果想用一级学科的话需要替换学科名，把逗号后边的二级删掉

['Engineering, Electrical & Electronic',
 'Linguistics',
 'Mathematical & Computational Biology',
 'Computer Science, Theory & Methods',
 'Psychology',
 'Biodiversity Conservation',
 'Computer Science, Hardware & Architecture',
 'Psychology, Mathematical',
 'Psychiatry',
 'Education, Scientific Disciplines',
 'Biochemical Research Methods',
 'Biotechnology & Applied Microbiology',
 'Computer Science, Information Systems',
 'Medical Informatics',
 'Health Care Sciences & Services',
 'Pharmacology & Pharmacy',
 'Statistics & Probability',
 'Computer Science, Artificial Intelligence',
 'Computer Science, Interdisciplinary Applications',
 'Education & Educational Research',
 'Psychology, Educational',
 'Computer Science, Cybernetics',
 'Automation & Control Systems',
 'Medicine, General & Internal',
 'Management',
 'Information Science & Library Science',
 'Mathematics, Applied',
 'Transportation Science & Technology',
 'Computer Science, Software Engineering',
 'Ecology',
 'Psychology, Ex

In [16]:
xueke_frame = pd.DataFrame(np.zeros([len(xueke_list_unique),len(xueke_list_unique)]),columns=xueke_list_unique,index=xueke_list_unique)
for wc in df_wc.WC:
    wc_list = wc.split('; ')
    for i in range(len(wc_list)-1):
        xueke_frame.loc[wc_list[i]][wc_list[i+1]]=xueke_frame.loc[wc_list[i]][wc_list[i+1]]+1

for j in range(len(xueke_frame)):#对角线都为0
    xueke_frame.iloc[i,i]=0

In [17]:
g = nx.Graph()

In [18]:
for i in range(len(xueke_list_unique)-1):
    xueke = xueke_list_unique[i]
    for j in range(i+1,len(xueke_list_unique)):
        w=0
        xueke2 = xueke_list_unique[j]
        w = xueke_frame.loc[xueke][xueke2]+xueke_frame.loc[xueke2][xueke]
        g.add_edge(xueke,xueke2,weight=w)

In [19]:
# nx.draw(g)
# plt.show()
nx.write_pajek(g,"D:/software/0install package/VOSviewer/web tab.net")
#将net文件导入vosviewer